移除卡幀圖片並轉換為 ReID 資料集

In [ ]:
import cv2
import os
import shutil
from tqdm import tqdm
import numpy as np


# 指定輸入文件夾、輸出文件夾路徑
input_folder = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2\images"  # r"your_path\trainv2\images"
output_folder = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling\images"    # r"your_path\trainv2_Frame_Skipping_Handling\images" 

threshold = 50  # 根據需要調整閾值
num_to_compare = 1  # 只與當前圖片後面的1張圖片比較相似度

# 計算兩張圖片的均方誤差（MSE）
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

# 處理文件夾中的圖片
def process_images_in_folders(input_folder, output_folder, threshold, num_to_compare=1):
    # 創建輸出文件夾
    os.makedirs(output_folder, exist_ok=True)
    
    # 獲取輸入文件夾中的所有子文件夾（每個子文件夾都包含要處理的圖片）
    subfolders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    # 使用 tqdm 顯示總進度條
    with tqdm(total=len(subfolders), desc="Processing Subfolders") as pbar:
        # 遍歷每個子文件夾
        for subfolder in subfolders:
            # 創建當前子文件夾對應的輸出路徑
            relative_subfolder = os.path.relpath(subfolder, input_folder)  # 相對路徑
            output_subfolder = os.path.join(output_folder, relative_subfolder)
            os.makedirs(output_subfolder, exist_ok=True)
            
            # 獲取當前子文件夾的所有圖片路徑
            image_paths = [os.path.join(subfolder, f) for f in os.listdir(subfolder) if f.endswith('.jpg')]
            num_images = len(image_paths)
            
            # 使用 tqdm 顯示當前子文件夾中的圖片處理進度
            for img_path in tqdm(image_paths, desc=f"Processing {relative_subfolder}", leave=False):
                img_name = os.path.basename(img_path)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (100, 100))  # 調整圖片尺寸，加快計算速度
                output_path = os.path.join(output_subfolder, img_name)
                
                # 標記是否保留當前圖片
                keep_image = True
                
                # 加載後續圖片並與當前圖片比較相似度
                img_index = image_paths.index(img_path)
                for j in range(img_index + 1, min(img_index + num_to_compare + 1, num_images)):
                    next_img_path = image_paths[j]
                    next_img = cv2.imread(next_img_path)
                    next_img = cv2.resize(next_img, (100, 100))  # 調整圖片尺寸，加快計算速度
                    mse_value = mse(img, next_img)
                    
                    if mse_value < threshold:
                        keep_image = False
                        break
                
                # 如果符合條件且是第一張圖片，則複製到輸出文件夾
                if keep_image:
                    # 檢查輸出文件夾中是否已存在同名文件
                    if not os.path.exists(output_path):
                        shutil.copy(img_path, output_path)
            
            # 更新進度條
            pbar.update(1)

    print("圖片處理完成！")

# 處理圖片
process_images_in_folders(input_folder, output_folder, threshold, num_to_compare)


In [ ]:
import os
import shutil
from tqdm import tqdm

# 定義主圖像文件夾、主標籤文件夾和目標主文件夾的路徑
main_image_folder = r'E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling\images'   # r"your_path\trainv2_Frame_Skipping_Handling\images"
main_label_folder = r'E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2\labels'                           # r"your_path\trainv2\labels"
output_main_folder = r'E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling\labels'  # r"your_path\trainv2_Frame_Skipping_Handling\labels"

# 確保輸出主文件夾存在，如果不存在則創建
os.makedirs(output_main_folder, exist_ok=True)

def copy_files_with_structure(source_folder, destination_folder):
    # 遍歷源文件夾中的所有子文件夾和文件
    for root, dirs, files in os.walk(source_folder):
        # 構建目標文件夾中對應的子文件夾路徑
        relative_root = os.path.relpath(root, source_folder)
        dest_root = os.path.join(destination_folder, relative_root)

        # 確保目標子文件夾存在，如果不存在則創建
        os.makedirs(dest_root, exist_ok=True)

        # 複製源文件夾中的文件到目標文件夾中的相應位置
        for file in files:
            source_file_path = os.path.join(root, file)
            dest_file_path = os.path.join(dest_root, file)
            shutil.copy(source_file_path, dest_file_path)


# 獲取主標籤文件夾中所有的子文件夾（每個子文件夾代表一個類別）
subfolders = [f.path for f in os.scandir(main_label_folder) if f.is_dir()]

# 遍歷每個子文件夾（每個子文件夾對應一個類別）
for subfolder in subfolders:
    class_name = os.path.basename(subfolder)  # 類別名稱（子文件夾名稱）

    # 獲取當前類別對應的主圖像文件夾和主標籤文件夾的路徑
    class_image_folder = os.path.join(main_image_folder, class_name)
    class_label_folder = os.path.join(main_label_folder, class_name)

    # 確保當前類別對應的主圖像文件夾存在
    if os.path.exists(class_image_folder):
        # 創建當前類別對應的目標輸出文件夾路徑
        output_class_folder = os.path.join(output_main_folder, class_name)
        os.makedirs(output_class_folder, exist_ok=True)

        # 獲取當前類別對應的主圖像文件夾中所有的圖像文件名
        image_files = [f for f in os.listdir(class_image_folder) if f.endswith('.jpg') or f.endswith('.png')]

        # 遍歷當前類別對應的主標籤文件夾中的所有標籤文件
        label_files = [f for f in os.listdir(class_label_folder) if f.endswith('.txt')]

        # 遍歷當前類別對應的主標籤文件夾中的每個標籤文件
        for label_file in tqdm(label_files, desc=f"Processing {class_name}"):
            label_name = os.path.splitext(label_file)[0]  # 提取標籤文件名（不含擴展名）

            # 假設標籤文件名與圖像文件名相同，擴展名為 .txt
            image_file = f'{label_name}.jpg'  # 假設圖像文件名與標籤文件名對應
            image_path = os.path.join(class_image_folder, image_file)

            # 檢查當前標籤文件是否與圖像文件名相對應
            if os.path.exists(image_path):
                # 如果相對應，複製標籤文件到目標輸出文件夾中的對應類別文件夾
                shutil.copy(os.path.join(class_label_folder, label_file), os.path.join(output_class_folder, label_file))

print("文件處理完成！")


In [2]:
import os
import cv2
import glob
import argparse

from tqdm import tqdm


def convert_bbox(args, data):
    img_height, img_width = args.imgsz
    
    # Normalized bbox to Absolute
    bb_width = float(data[3]) * img_width
    bb_height = float(data[4]) * img_height
    bb_left = float(data[1]) * img_width - bb_width / 2
    bb_top = float(data[2]) * img_height - bb_height / 2
    track_id = data[5].split('\n')[0]
    
    x1, y1, x2, y2 = bb_left, bb_top, bb_left + bb_width, bb_top + bb_height

    return int(x1), int(y1), int(x2), int(y2), int(track_id)


def make_parser():
    parser = argparse.ArgumentParser("AICUP ReID dataset")

    parser.add_argument("--data_path", default="trainv2_Frame_Skipping_Handling", help="path to AICUP train data")
    parser.add_argument("--save_path", default="REID_datasets/test", help="Path to save the MOT-ReID dataset")
    parser.add_argument('--imgsz', type=tuple, default=(720, 1280), help='img size, (height, width)')
    parser.add_argument('--train_ratio', type=float, default=1.0, help='The ratio of the train set when splitting the train set and the test set')

    return parser


def main(args):

    # Create folder for outputs
    save_path = os.path.join(args.save_path, 'AICUP-ReID')
    os.makedirs(save_path, exist_ok=True)

    train_save_path = os.path.join(save_path, 'bounding_box_train')
    os.makedirs(train_save_path, exist_ok=True)
    test_save_path = os.path.join(save_path, 'bounding_box_test')
    os.makedirs(test_save_path, exist_ok=True)
    
    # Split train/test set
    total_files = sorted(glob.glob(os.path.join(args.data_path, 'images', '*')))
    total_count = len(total_files)
    train_count = int(total_count * args.train_ratio)
    train_files = total_files[:train_count]
    
    for time_path in total_files:
        for img_path in tqdm(glob.glob(os.path.join(time_path, '*')), desc=f'convert {time_path}'):
            path_txt = img_path.split(os.sep)
            timestemp, img_name = path_txt[-2], path_txt[-1].split('.')[0]
            frame_id = img_name.split('_')[-1]
            img = cv2.imread(img_path)
            
            with open(os.path.join(args.data_path, 'labels', timestemp, f'{img_name}.txt'), 'r') as gt_file:
                for line in gt_file.readlines():
                    data = line.split(' ')
                    x1, y1, x2, y2, track_id = convert_bbox(args, data)
                    patch = img[y1:y2, x1:x2, :]
                    
                    # Resize patch to 256x256
                    patch_resized = cv2.resize(patch, (256, 256), interpolation=cv2.INTER_CUBIC)
                    
                    # TrackID_TimeStemp_FrameID_acc_data.bmp
                    file_name = f'{str(track_id).zfill(7)}_{timestemp.replace("_", "-")}_{str(frame_id).zfill(7)}_acc_data.bmp'

                    if time_path in train_files:
                        cv2.imwrite(os.path.join(train_save_path, file_name), patch_resized)
                    else:
                        cv2.imwrite(os.path.join(test_save_path, file_name), patch_resized)


if __name__ == "__main__":

    class Args:
        data_path = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling"  # r"your_path\trainv2_Frame_Skipping_Handling"
        save_path = r"E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\REID_datasets\day-night_card_detection_ESRGAN"      # r"your_path\REID_datasets\day-night_card_detection_ESRGAN"
        imgsz = (720, 1280)
        train_ratio = 1.0
    
    args = Args()
    main(args)


convert E:\AI_CUP\AI-driven_Future\AICUP_Baseline_BoT-SORT-main\trainv2_Frame_Skipping_Handling\images\0902_150000_151900:  48%|████▊     | 905/1876 [00:22<00:23, 40.78it/s]


KeyboardInterrupt: 